In [1]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from collections import Counter

In [2]:
torch.manual_seed(42)

In [3]:
class SVR(nn.Module):
    def __init__(self):
        super(SVR,self).__init__()
        self.fc1 = nn.Linear(11, 1)
        self.fc2 = nn.Linear(1, 1)
        
    def forward(self,x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

In [4]:
model=SVR()
# optimizer=torch.optim.SGD(model.parameters(),lr=0.0005)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [ ]:
# loss加权重，减少数据标签数量差异的影响

In [5]:
wine_data = pd.read_csv('../data/winequality-white.csv',sep=';')
wine_data = wine_data.values
x = wine_data[:,:11]
x_s = preprocessing.StandardScaler().fit(x)
x = x_s.transform(x)
y = wine_data[:,11]

In [6]:
Counter(y)

Counter({6.0: 2198, 5.0: 1457, 7.0: 880, 8.0: 175, 4.0: 163, 3.0: 20, 9.0: 5})

In [7]:
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.3)

In [8]:
X = torch.from_numpy(x_train).type(torch.FloatTensor)
Y = torch.from_numpy(y_train).type(torch.FloatTensor)
X_t = torch.from_numpy(x_test).type(torch.FloatTensor)
Y_t = torch.from_numpy(y_test).type(torch.FloatTensor)

In [9]:
def calc_acc(predicted, real):
    precise_score = torch.round(predicted).detach().numpy()
    return accuracy_score(precise_score, real.detach().numpy())

In [10]:
TOTAL_EPOCH = 10000
for epoch in range(TOTAL_EPOCH):
    optimizer.zero_grad()
    predicted = model(X)
    loss = criterion(predicted, Y)
    loss.backward()
    optimizer.step()
    if epoch % 200 == 0:
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, TOTAL_EPOCH, loss))
        print('train_score: {:.2f}'.format(calc_acc(predicted, Y)))
        with torch.no_grad():
            t_predicted = model(X_t)
            print('test_score: {:.2f}'.format(calc_acc(t_predicted, Y_t)))
            print('test_loss: {:.4f}\n'.format(criterion(t_predicted, Y_t)))
        

/home/zfj/anaconda3/envs/torch1.4/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([3428])) that is different to the input size (torch.Size([3428, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/zfj/anaconda3/envs/torch1.4/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1470])) that is different to the input size (torch.Size([1470, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch [1/10000], loss:31.8353
train_score: 0.00
test_score: 0.00
test_loss: 31.7506

epoch [201/10000], loss:25.8244
train_score: 0.00
test_score: 0.00
test_loss: 25.6832

epoch [401/10000], loss:19.3271
train_score: 0.02
test_score: 0.02
test_loss: 19.2255

epoch [601/10000], loss:15.9201
train_score: 0.06
test_score: 0.06
test_loss: 15.9147

epoch [801/10000], loss:14.2131
train_score: 0.08
test_score: 0.08
test_loss: 14.2516

epoch [1001/10000], loss:12.7084
train_score: 0.08
test_score: 0.09
test_loss: 12.7693

epoch [1201/10000], loss:11.0119
train_score: 0.09
test_score: 0.10
test_loss: 11.1564

epoch [1401/10000], loss:8.1237
train_score: 0.13
test_score: 0.10
test_loss: 8.2710

epoch [1601/10000], loss:4.5625
train_score: 0.19
test_score: 0.18
test_loss: 4.4566

epoch [1801/10000], loss:1.7382
train_score: 0.30
test_score: 0.31
test_loss: 1.6311

epoch [2001/10000], loss:0.8382
train_score: 0.45
test_score: 0.44
test_loss: 0.8430

epoch [2201/10000], loss:0.7864
train_score: 0.

In [25]:
with torch.no_grad():
    t_predicted = model(X_t)
    result = torch.round(t_predicted).detach().numpy()[:,0]
    print(Counter(result))
    
# Counter({6.0: 1118, 5.0: 351, 8.0: 1})
# Counter({6.0: 1466, 7.0: 3, 8.0: 1})

Counter({6.0: 1466, 7.0: 3, 8.0: 1})


In [23]:
with torch.no_grad():
    t_predicted = model(X_t)
    print('test_score: {:.2f}\n'.format(calc_acc(t_predicted, Y_t)))

test_score: 0.41



In [11]:
# adam 11-11-1 lr=0.001  loss-0.792  train-0.44  test-0.44
# SGD-relu 11-11-1 lr=0.0005  loss-0.792  train-0.44  test-0.44
# adam 11-6-1 lr=0.001  loss-0.7725  train-0.447  test-0.45
# adam 11-4-1 lr=0.001  loss-0.7942  train-0.45  test-0.45
# adam-relu 11-1-1 lr=0.001  loss-0.8052  train-0.44  test-0.47

# SGD 11-1 lr=0.0005  loss-0.7909  train-0.45  test-0.45